In [ ]:
import pandas as pd
import joblib

In [ ]:
# Load test data
test_df = pd.read_csv("C:/Users/ANKITA UPADHAYAY/Downloads/hacktest.csv")

In [ ]:
# Load saved models and encoders
model = joblib.load('logistic_model.pkl')
scaler = joblib.load('scaler.pkl')
poly = joblib.load('poly.pkl')
pca = joblib.load('pca.pkl')
le = joblib.load('label_encoder.pkl')

In [ ]:
# NDVI columns
ndvi_cols = [col for col in test_df.columns if col.endswith('_N')]

In [ ]:
# Interpolate and smooth NDVI values
test_df[ndvi_cols] = test_df[ndvi_cols].interpolate(method='linear', axis=1, limit_direction='both')
test_df[ndvi_cols] = test_df[ndvi_cols].T.rolling(window=3, min_periods=1).mean().T

In [ ]:
# Add same features
test_df['ndvi_mean'] = test_df[ndvi_cols].mean(axis=1)
test_df['ndvi_std'] = test_df[ndvi_cols].std(axis=1)
test_df['ndvi_max'] = test_df[ndvi_cols].max(axis=1)
test_df['ndvi_min'] = test_df[ndvi_cols].min(axis=1)
test_df['ndvi_range'] = test_df['ndvi_max'] - test_df['ndvi_min']
test_df['ndvi_skew'] = test_df[ndvi_cols].skew(axis=1)
test_df['ndvi_kurtosis'] = test_df[ndvi_cols].kurtosis(axis=1)

In [ ]:
# Save IDs for submission
test_ids = test_df['ID']
X_test = test_df.drop(columns=['ID'])

In [ ]:
# Apply scaling, polynomial transform, PCA
X_test_scaled = scaler.transform(X_test)
X_test_poly = poly.transform(X_test_scaled)
X_test_pca = pca.transform(X_test_poly)

In [ ]:
# Predict
preds_encoded = model.predict(X_test_pca)
preds = le.inverse_transform(preds_encoded)


In [ ]:
# Submission file
submission = pd.DataFrame({'ID': test_ids, 'class': preds})
submission.to_csv("submission5_logistic_ndvi.csv", index=False)
print("✅ Submission file 'submission5_logistic_ndvi.csv' saved successfully.")